## Stock Price Predictor
### 1. Finacial Data
- We need financial data (yahoo api is a great place)
- We need to clean that data
- Put in the missing value
### 2. Selecting a model
There is are several choices like 
- Support Vector Machine
- Linear Regression
- LSTM option is also there (**Probably use this one**)
### 3. Train and test
- Divide the data into two groups(Train and test)
- Test the accuracy of the the model

In [24]:
import yfinance as yf
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
# Importing Long short term model

In [9]:
data = yf.download("GOOGL" , start='2022-01-01' , end='2025-01-01')
print(data.columns)
X= data.drop(columns=[('Close' , 'GOOGL')])
y = data[('Close' , 'GOOGL')]
print(X.shape)
print(y.shape)

[*********************100%***********************]  1 of 1 completed

MultiIndex([( 'Close', 'GOOGL'),
            (  'High', 'GOOGL'),
            (   'Low', 'GOOGL'),
            (  'Open', 'GOOGL'),
            ('Volume', 'GOOGL')],
           names=['Price', 'Ticker'])
(753, 4)
(753,)


In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X , y ,test_size=0.2 ,train_size=0.8, random_state=0)

model= LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test,y_pred)

print("Mean Squared Error : ",mse)

Mean Squared Error :  0.9261329399632023


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler


price = data['Close'] # Only the closing price is selected

# Normalizing the data for the LSTM
scalar = MinMaxScaler(feature_range=(0,1))
prices_scaled = scalar.fit_transform(price)

In [26]:
def create_sequences(data,time_step):
    X , y = [] , []
    for i in range(len(data)-time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X) , np.array(y)
    
time_step = 60
X, y = create_sequences(prices_scaled , time_step)

# Reshaping the data for LSTM 
X = X.reshape(X.shape[0],X.shape[1],1)


In [39]:
from tensorflow.keras.optimizers import Adam
split = int(len(data)*0.8)
X1_train,X1_test = X[:split] , X[split:]
y1_train,y1_test = y[:split] , y[split:]

modelL = Sequential([
    LSTM(50,return_sequences=True,input_shape=(time_step,1)),
    Dropout(0.2), # Prevents Overfitting

    LSTM(50,return_sequences=False),
    Dropout(0.2),

    Dense(25),
    Dense(1)
])

modelL.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')

c:\Users\basit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
history = modelL.fit(X1_train,y1_train,epochs=50,batch_size=32, validation_data=(X1_test,y1_test))

Epoch 1/50


19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 121ms/step - loss: 0.0691 - val_loss: 0.0449
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - loss: 0.0118 - val_loss: 0.0162
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 0.0054 - val_loss: 0.0066
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - loss: 0.0043 - val_loss: 0.0051
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - loss: 0.0039 - val_loss: 0.0064
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0035 - val_loss: 0.0050
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0039 - val_loss: 0.0033
Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - loss: 0.0030 - val_loss: 0.0052
Epoch 12/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0036 - v

In [43]:
y_pred1 = modelL.predict(X1_test)

mse = mean_squared_error(y_pred1,y1_test)
print("MSE for LSTM is: " , mse) # Outputs a very good score

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
MSE for LSTM is:  0.0018574046920053885
